In [1]:
import pandas as pd
from datetime import date, timedelta
from random import randint
from time import sleep
import os
import numpy as np
import pymysql
from sqlalchemy import create_engine
import joblib
import requests
from bs4 import BeautifulSoup
import pulp

## TODO
[x] Change`end_date = start_date + timedelta(days=9)` back to `end_date = start_date + timedelta(days=6)`<br>
[x] Change `today + timedelta(days=12)` to `today` on season start date <br>
[ ] Update hardcoded year in "Win % for SoS Calc" after season start <br>
[x] Confirm where new 2024/2025 data will come from/ live (CSV, DB etc) <br> 

In [2]:
## Variables
year = 2025

# Get today's date
today = date.today()
season_start_date = '2024-10-04'

In [3]:
username = "root"
password = "Sp1d3rman"
host = "localhost"
port = "3306"
database = "nhl_optimizer"

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

In [4]:
query = f"""
SELECT *
FROM player_data
WHERE Date >='{season_start_date}'
"""

df = pd.read_sql(query,engine)

In [5]:
# Clean the data
df['Date'] = pd.to_datetime(df['Date'])
df = df.drop_duplicates()

df = df.sort_values(by=['Player','Date'])

In [6]:
# Compute rolling aggregates

# Define the number of games for the rolling window
n_games = 5 # adjust as needed

# Group by 'Player' and apply rolling calculations
grouped = df.groupby('Player',as_index=False,group_keys=False)

stats = ['GP','TOI', 'TOI/GP', 'Goals/60',
       'Total Assists/60', 'First Assists/60', 'Second Assists/60',
       'Total Points/60', 'IPP', 'Shots/60', 'SH%', 'ixG/60', 'iCF/60',
       'iFF/60', 'iSCF/60', 'iHDCF/60', 'Rush Attempts/60',
       'Rebounds Created/60', 'PIM/60', 'Total Penalties/60', 'Minor/60',
       'Major/60', 'Misconduct/60', 'Penalties Drawn/60', 'Giveaways/60',
       'Takeaways/60', 'Hits/60', 'Hits Taken/60', 'Shots Blocked/60',
       'Faceoffs Won/60', 'Faceoffs Lost/60','Faceoffs %']

for stat in stats:
    df[stat] = grouped[stat].apply(lambda x: x.rolling(window=n_games,min_periods=1).mean())
    
latest_stats = df.groupby('Player').tail(1)


final_df = latest_stats[['Player','Team','Position'] + stats].reset_index(drop=True)

### Player Point Value Data

In [7]:
pv_df = pd.read_csv('nhl_players.csv')

## Load and Process Models

In [8]:
# Load the models
loaded_goals_pipeline = joblib.load('goals_model.pkl')
loaded_assists_pipeline = joblib.load('assists_model.pkl')

# Read and process the data
# season_totals = pd.read_csv('season_totals_24.csv',index_col=0)
# season_totals.drop(columns=["Goals/60","Total Assists/60"], inplace=True)

season_totals = final_df.drop(columns=['Goals/60','Total Assists/60'])


# # Predictions
goal_predictions = loaded_goals_pipeline.predict(season_totals)
assist_predictions = loaded_assists_pipeline.predict(season_totals)

## Functions

In [9]:
from datetime import date, timedelta

def fetch_game_data(year: int):
    """
    Fetches the dates of NHL games and teams involved for a given year.
    
    Parameters:
    - year: The year for which you want to fetch game data.

    Returns:
    - DataFrame with dates, visitor, and home teams.
    """
    
    url = f"https://www.hockey-reference.com/leagues/NHL_{year}_games.html"
    dfs = pd.read_html(url)
    df = dfs[0]
    
    # Convert the "Date" column to datetime.date format
    df["Date"] = pd.to_datetime(df["Date"]).dt.date

    # Extract the relevant columns
    game_data = df[["Date", "Visitor", "Home"]]

    return game_data

def filter_dates_for_week(dates, start_date):
    """
    Filters the provided dates for those that fall within the week starting at start_date.
    
    Parameters:
    - dates: List of dates.
    - start_date: The starting date of the week. 

    Returns:
    - A list of dates that fall within the desired week.
    """
    # Convert start_date to a date object if it's a string
    if isinstance(start_date, str):
        start_date = date.fromisoformat(start_date)
    
    # Calculate the end date of the week
    end_date = start_date + timedelta(days=6)

    # Filter the dates
    week_dates = [d for d in dates if start_date <= d <= end_date]
    
    return week_dates

def opponents_for_team_for_week(year, start_date):
    """
    Fetches the opponents each team faces within a specified week.
    
    Parameters:
    - year: The year for which you want to check.
    - start_date: The starting date of the week.

    Returns:
    - A dictionary with teams as keys and lists of opponents as values.
    """
    # Fetch game data for the year
    game_data = fetch_game_data(year)
    
    # Filter the game data for the desired week
    week_dates = filter_dates_for_week(game_data["Date"].tolist(), start_date)
    week_games = game_data[game_data["Date"].isin(week_dates)]
    
    # Prepare opponent information
    visitor_opponents = week_games.set_index("Visitor")["Home"].to_dict()
    home_opponents = week_games.set_index("Home")["Visitor"].to_dict()
    
    opponents = {}
    for team in set(week_games["Visitor"].tolist() + week_games["Home"].tolist()):
        opponents[team] = list(set([visitor_opponents.get(team, "")] + [home_opponents.get(team, "")]))

    return opponents


#-------#

def calculate_weekly_team_multipliers(year, start_date, points_df):
    """
    Calculate the weekly multipliers for teams based on their opponents for a given week.
    """
    
    # Fetch the opponents for each team for the week
    games_week = opponents_for_team_for_week(year, start_date)

    weekly_multipliers = {}
    for team, opponents in games_week.items():
        multipliers = []
        for opponent in opponents:
            multiplier_row = points_df[points_df['Team Name'] == opponent]
            if not multiplier_row.empty:
                multipliers.append(multiplier_row['multiplier'].iloc[0])
            else:
                print(f"Warning: Couldn't find multiplier for opponent {opponent}. Skipping...")

        weekly_multipliers[team] = sum(multipliers) / len(multipliers) if multipliers else 1  # Defaulting to 1 if no games

    return weekly_multipliers

# Now, you can create a new column in your players DataFrame to store the composite multiplier for the week:


def games_count_for_team_for_week(year, start_date):
    """
    Fetches the number of games each team plays within a specified week.
    
    Parameters:
    - year: The year for which you want to check.
    - start_date: The starting date of the week.

    Returns:
    - A dictionary with teams as keys and the number of games they play that week as values.
    """
    
    # Fetch game data for the year
    game_data = fetch_game_data(year)
    
    # Filter the game data for the desired week
    week_dates = filter_dates_for_week(game_data["Date"].tolist(), start_date)
    week_games = game_data[game_data["Date"].isin(week_dates)]
    
    # Count the number of games for each team
    visitor_counts = week_games["Visitor"].value_counts().to_dict()
    home_counts = week_games["Home"].value_counts().to_dict()
    
    total_counts = {}
    for team in set(list(visitor_counts.keys()) + list(home_counts.keys())):
        total_counts[team] = visitor_counts.get(team, 0) + home_counts.get(team, 0)

    return total_counts

## Team Name Dict

In [10]:
# TEAM NAMES
team_abbreviations = {
 'Anaheim Ducks':'ANA',
 'Boston Bruins':'BOS',
 'Buffalo Sabres':'BUF',
 'Calgary Flames':'CGY',
 'Carolina Hurricanes':'CAR',
 'Chicago Blackhawks':'CHI',
 'Colorado Avalanche':'COL',
 'Columbus Blue Jackets':'CBJ',
 'Dallas Stars':'DAL',
 'Detroit Red Wings':'DET',
 'Edmonton Oilers':'EDM',
 'Florida Panthers':'FLA',
 'Los Angeles Kings':'L.A',
 'Minnesota Wild':'MIN',
 'Montreal Canadiens':'MTL',
 'Nashville Predators':'NSH',
 'New Jersey Devils':'N.J',
 'New York Islanders':'NYI',
 'New York Rangers':'NYR',
 'Ottawa Senators':'OTT',
 'Philadelphia Flyers':'PHI',
 'Pittsburgh Penguins':'PIT',
 'San Jose Sharks':'S.J',
 'Seattle Kraken':'SEA',
 'St. Louis Blues':'STL',
 'Tampa Bay Lightning':'T.B',
 'Toronto Maple Leafs':'TOR',
 'Utah Hockey Club':'UTA',
 'Vancouver Canucks':'VAN',
 'Vegas Golden Knights':'VGK',
 'Washington Capitals':'WSH',
 'Winnipeg Jets':'WPG'
}

## Injuries

In [11]:
# Fetch the webpage content
url = "https://www.cbssports.com/nhl/injuries/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Lists to hold our parsed data
teams = []
injury_dataframes = []

# Iterate over each TableBaseWrapper
for wrapper in soup.find_all('div', class_='TableBaseWrapper'):
    
    # Find team name
    team_name_tag = wrapper.find('span', class_='TeamName')
    if team_name_tag:
        team_name = team_name_tag.get_text(strip=True)
        teams.append(team_name)

        # Find the injury table for the team
        table = wrapper.find('table', class_='TableBase-table')
        if table:
            # Use pandas to read the table
            df = pd.read_html(str(table))[0]
            df['Team'] = team_name  # Add a column for the team name
            injury_dataframes.append(df)

# Concatenate all dataframes
injuries_df = pd.concat(injury_dataframes, ignore_index=True)

#Clean the player name up


def clean_player_name(name):
    # Split the name using spaces to identify potential last names
    parts = name.split()

    # The last word (or the two last words in the case of names like "van Riemsdyk") is likely our surname
    surname = parts[-2] + " " + parts[-1] if len(parts) > 2 and len(parts[-2]) <= 3 else parts[-1]

    # Split by surname and get the second part
    full_name = name.split(surname, 1)[1].strip()

    return full_name if full_name else surname

injuries_df['Player'] = injuries_df['Player'].apply(clean_player_name)

## Variables

In [12]:
## VARIABLES

## Updated to match new scoring rules for Office Pools
fantasy_goals = 2
fantasy_assists = 1
goaltending_win = 2
ot_so_loss = 0
shutout = 2

# all_dates = fetch_game_dates(year)
# week_dates = filter_dates_for_week(all_dates, today)

#### Win % for SoS Calc

In [13]:
## GATHER WIN % TO USE FOR SOS CALC

dfs = pd.read_html("https://www.hockey-reference.com/leagues/NHL_2024_standings.html", index_col=0)
east_df = dfs[0].reset_index().rename(columns={'index': 'Team Name'})
west_df = dfs[1].reset_index().rename(columns={'index': 'Team Name'})

east_df = east_df.drop([0, 9], axis=0).reset_index(drop=True)
west_df = west_df.drop([0, 9], axis=0).reset_index(drop=True)
east_df['Team Name'] = east_df['Team Name'].str.replace('*', '', regex=False)
west_df['Team Name'] = west_df['Team Name'].str.replace('*', '', regex=False)


east_df = east_df[['Team Name', 'PTS%']]
west_df = west_df[['Team Name', 'PTS%']]

points_df = pd.concat([east_df, west_df], ignore_index=True)
points_df['PTS%'] = points_df['PTS%'].astype(float)


points_df['Abbreviation'] = points_df['Team Name'].map(team_abbreviations)

## Build Reference DataFrame

In [14]:
# Create dataframe for reference

player_df = pd.DataFrame({
    'Team':season_totals['Team'].values,
    'Position':season_totals['Position'].values,
    'Player':season_totals['Player'].values,
    'Projected Goals/60':goal_predictions,
    'Projected Assists/60':assist_predictions,
    'Projected Goals (TOI)':(season_totals['TOI/GP']/60) * goal_predictions,
    'Projected Assists (TOI)':(season_totals['TOI/GP']/60) * assist_predictions,
})

player_df = player_df.merge(points_df, left_on='Team', right_on='Abbreviation', how='left')

player_df = player_df.round(2).sort_values(by='Projected Goals (TOI)', ascending=False)
player_df['Team'] = player_df['Team Name'].str.split(',').str[0].str.strip()

player_df = player_df[['Team','Abbreviation','Player', 'Position','Projected Goals (TOI)', 'Projected Assists (TOI)']]

player_df = player_df.merge(pv_df[["Team","Player","Position","pv"]], 
                     on=["Team",'Player',"Position"], 
                     how='left')

# 3. Select and rename the desired columns
player_df = player_df.rename(columns={
    'Team_x': 'Old Team',
    'Team_y': 'Team',
})

player_df = player_df[['Team', 'Abbreviation', 'Player', 'Position', 'Projected Goals (TOI)', 'Projected Assists (TOI)']]

# Cleaning the team name (remove asterisks and keep only abbreviation, if needed)
points_df['Team'] = points_df['Team Name']

# Calculating multiplier based on points percentage
points_df['multiplier'] = points_df['PTS%'].apply(lambda x: 0.5 / x)

In [15]:
weekly_multipliers = calculate_weekly_team_multipliers(year, today, points_df)
games_this_week = games_count_for_team_for_week(year, today)


# Map this to your players DataFrame
player_df['weekly_multiplier'] = player_df['Team'].map(weekly_multipliers)
player_df['games_this_week'] = player_df['Team'].map(games_this_week)

In [16]:
player_df['projected_goals'] = player_df['Projected Goals (TOI)'] * player_df['weekly_multiplier'] * player_df['games_this_week']
player_df['projected_assists'] = player_df['Projected Assists (TOI)'] * player_df['weekly_multiplier'] * player_df['games_this_week']

# Factoring in fantasy points value for goals and assists
player_df['proj_fantasy_pts'] = (player_df['projected_goals'] * fantasy_goals) + (player_df['projected_assists'] * fantasy_assists)

# Create simple F or D position
player_df.loc[player_df['Position'] != 'D', 'Position'] = 'F'
player_df.head(10)

,Team,Abbreviation,Player,Position,Projected Goals (TOI),Projected Assists (TOI),weekly_multiplier,games_this_week,projected_goals,projected_assists,proj_fantasy_pts
0,Tampa Bay Lightning,T.B,Nikita Kucherov,F,0.58,0.53,0.836120,2.0,0.969900,0.886288,2.826087
1,Colorado Avalanche,COL,Mikko Rantanen,F,0.58,0.71,1.388889,4.0,3.222222,3.944444,10.388889
2,Nashville Predators,NSH,Jonathan Marchessault,F,0.57,0.79,0.844772,3.0,1.444560,2.002110,4.891231
3,Colorado Avalanche,COL,Nathan MacKinnon,F,0.57,0.60,1.388889,4.0,3.166667,3.333333,9.666667
4,New Jersey Devils,N.J,Timo Meier,F,0.55,0.41,1.050420,4.0,2.310924,1.722689,6.344538
5,Toronto Maple Leafs,TOR,Auston Matthews,F,0.54,0.52,0.827815,2.0,0.894040,0.860927,2.649007
6,Nashville Predators,NSH,Filip Forsberg,F,0.52,0.34,0.844772,3.0,1.317845,0.861668,3.497357
7,Winnipeg Jets,WPG,Mark Scheifele,F,0.52,0.47,1.742160,2.0,1.811847,1.637631,5.261324
8,Buffalo Sabres,BUF,Tage Thompson,F,0.51,0.47,0.994469,3.0,1.521537,1.402201,4.445275
9,Columbus Blue Jackets,CBJ,Zach Werenski,D,0.50,0.65,0.871717,3.0,1.307575,1.699848,4.314998


In [17]:
# Now, merge on 'Player'
merged_df = pd.merge(player_df, injuries_df[['Player', 'Injury Status']], 
                     on='Player', 
                     how='left')

# Set a new column to indicate if a player is injured or not
merged_df['Injured'] = ~merged_df['Injury Status'].isnull()

# Set the projected fantasy points to 0 for injured players
merged_df['proj_fantasy_pts'] = np.where(merged_df['Injured'], 0, merged_df['proj_fantasy_pts'])

In [18]:
projections_df = merged_df[['Team','Injured','Player','Position','games_this_week','proj_fantasy_pts']]

## Add Team Goaltending Calcs

In [19]:
def estimate_team_goaltending_points(weekly_multipliers, games_this_week, win_points, ot_loss_points, shutout_bonus, avg_ot_loss_freq=0.1, avg_shutout_freq=0.05):
    """
    Estimate the team goaltending points for the week using weekly multipliers, 
    number of games this week, and the fantasy points system.

    Returns:
    - A dictionary with team abbreviations as keys and a tuple of projected goaltending points and games_this_week as values.
    """

    goaltending_data = {}

    for team, multiplier in weekly_multipliers.items():
        games = games_this_week.get(team, 0)

        # Estimate wins based on multiplier (inverse relation)
        projected_wins = games / multiplier
        projected_ot_losses = games * avg_ot_loss_freq  # You can adjust or use team-specific data
        projected_shutouts = games * avg_shutout_freq   # You can adjust or use team-specific data

        total_points = (projected_wins * win_points) + (projected_ot_losses * ot_loss_points) + (projected_shutouts * shutout_bonus)
        
        goaltending_data[team] = (total_points,games)

    return goaltending_data

# Using the function
goaltending_data = estimate_team_goaltending_points(weekly_multipliers, games_this_week, goaltending_win, ot_so_loss, shutout)

# Convert to DataFrame
df = pd.DataFrame(goaltending_data.values(), index=goaltending_data.keys(), columns=['proj_fantasy_pts', 'games_this_week'])

# Reset index to have the teams as a separate column
df = df.reset_index().rename(columns={"index": "Team"})

# Add the position
df["Position"] = "G" 
df['Injured'] = False
df['Player'] = df['Team']


goalie_df = df[['Team','Injured','Player','Position','games_this_week','proj_fantasy_pts']]

## Final DataFrame Ready for Optimization

In [20]:
# Add goalie df to player df
final_df = pd.concat([projections_df, goalie_df], ignore_index=True).round(2)

## Add player values
final_df = pd.merge(final_df, pv_df[["Team","Player","Position","pv"]], 
                     on=['Team',"Position",'Player'], 
                     how='left')

final_df.loc[final_df['Position'] =='G','pv'] = 0

final_df = final_df.sort_values(by=["pv","proj_fantasy_pts"], ascending=False)
final_df.head(10)

,Team,Injured,Player,Position,games_this_week,proj_fantasy_pts,pv
5,Toronto Maple Leafs,False,Auston Matthews,F,2.0,2.65,13.250
3,Colorado Avalanche,False,Nathan MacKinnon,F,4.0,9.67,12.600
232,Edmonton Oilers,False,Connor McDavid,F,4.0,2.83,12.500
84,New York Rangers,False,Artemi Panarin,F,3.0,2.76,11.643
217,Vancouver Canucks,False,Elias Pettersson,F,2.0,1.10,11.600
379,Pittsburgh Penguins,False,Erik Karlsson,D,4.0,1.24,11.500
234,Toronto Maple Leafs,False,William Nylander,F,2.0,1.11,11.500
16,Boston Bruins,False,David Pastrnak,F,3.0,3.31,11.250
290,Buffalo Sabres,False,Rasmus Dahlin,D,3.0,1.91,11.000
32,Toronto Maple Leafs,True,John Tavares,F,2.0,0.00,11.000


## Optimize Lineup

In [21]:


## OPTIMIZATION RULES

"""
- 6 Forwards MUST be selected
- 4 Defense MUST be selected
- 2 team Goaltenders MUST be selected
- Players are given a salary from 1-5 points
- A team CANNOT go over the salary cap of 30 points
- At least 3 teams MUST be represented in the lineup
"""

MAX_COST = 63.00
NUM_FORWARDS = 6
NUM_DEFENSEMEN = 4 
NUM_GOALIES = 2

final_df = final_df.dropna(subset=['proj_fantasy_pts', 'pv'])

def select_best_team(df, max_cost, num_forwards, num_defensemen, num_goalies):
    # Create a linear programming problem
    prob = pulp.LpProblem("FantasyHockeyTeam", pulp.LpMaximize)

    # Create decision variables
    # Each player gets a binary decision variable
    player_vars = pulp.LpVariable.dicts("player", df.index, cat="Binary")

    # Objective: Maximize total fantasy points
    prob += pulp.lpSum(df['proj_fantasy_pts'][i] * player_vars[i] for i in df.index), "Total Fantasy Points"

    # Constraint: Stay within the salary cap
    prob += pulp.lpSum(df['pv'][i] * player_vars[i] for i in df.index) <= max_cost, "Total Salary Cost"

    # Positional constraints
    prob += pulp.lpSum(player_vars[i] for i in df[df['Position'] == 'F'].index) == num_forwards, "Number of Forwards"
    prob += pulp.lpSum(player_vars[i] for i in df[df['Position'] == 'D'].index) == num_defensemen, "Number of Defensemen"
    prob += pulp.lpSum(player_vars[i] for i in df[df['Position'] == 'G'].index) == num_goalies, "Number of Goalies"

    # Solve the problem
    prob.solve(pulp.PULP_CBC_CMD())
#      prob.solve(pulp.GLPK_CMD())


    # Extract the selected players
    selected_players = [i for i in df.index if player_vars[i].varValue == 1]
    best_team = df.loc[selected_players]

    return best_team

# Assuming your dataframe is named final_df, and you have defined constants for max_cost, num_forwards, etc.
best_team = select_best_team(final_df, MAX_COST, NUM_FORWARDS, NUM_DEFENSEMEN, NUM_GOALIES)


# Calculate total projected points and total cost for the chosen players from best_team
total_projected_points = sum(best_team.proj_fantasy_pts)
total_cost = sum(best_team.pv)

# Displaying the results

# Create a dataframe with the values
cost = pd.DataFrame({'total_proj_pts': [total_projected_points], 'total_cost': [total_cost]})


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/kellen/anaconda3/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/0cf6c9505953421fa393678df43bdaba-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/0cf6c9505953421fa393678df43bdaba-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 2457 RHS
At line 2462 BOUNDS
At line 2960 ENDATA
Problem MODEL has 4 rows, 497 columns and 962 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 84.6478 - 0.00 seconds
Cgl0004I processed model has 4 rows, 487 columns (487 integer (479 of which binary)) and 942 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.0625455
Cbc0038I Pass   1: suminf.    0.01483 (2) obj. -84.6073 iterations 4
Cbc0038I Pass   2: suminf.    0.00000 (0) obj. -77.55 

# Lineup

In [22]:
print(cost)
best_team.sort_values(by='Position')

   total_proj_pts  total_cost
0           84.29      62.286


,Team,Injured,Player,Position,games_this_week,proj_fantasy_pts,pv
66,New Jersey Devils,False,Dougie Hamilton,D,4.0,4.54,9.000
128,Colorado Avalanche,False,Josh Manson,D,4.0,4.50,4.500
197,Los Angeles Kings,False,Mikey Anderson,D,4.0,4.15,4.125
109,Dallas Stars,False,Thomas Harley,D,4.0,4.70,4.000
3,Colorado Avalanche,False,Nathan MacKinnon,F,4.0,9.67,12.600
1,Colorado Avalanche,False,Mikko Rantanen,F,4.0,10.39,9.250
33,New Jersey Devils,False,Jack Hughes,F,4.0,5.97,8.000
70,Colorado Avalanche,False,Casey Mittelstadt,F,4.0,6.89,5.750
11,Florida Panthers,False,Carter Verhaeghe,F,4.0,5.87,4.167
12,Dallas Stars,False,Wyatt Johnston,F,4.0,7.35,0.894


## Captains

In [23]:
# Find the top players by position based on 'proj_fantasy_pts'
top_players = best_team.loc[best_team.groupby('Position')['proj_fantasy_pts'].idxmax()]

top_players

,Team,Injured,Player,Position,games_this_week,proj_fantasy_pts,pv
109,Dallas Stars,False,Thomas Harley,D,4.0,4.70,4.00
1,Colorado Avalanche,False,Mikko Rantanen,F,4.0,10.39,9.25
554,Anaheim Ducks,False,Anaheim Ducks,G,4.0,10.83,0.00


In [24]:
data = [{'Team': key, 'Multiplier': value} for key, value in weekly_multipliers.items()]
multipliers = pd.DataFrame(data)
multipliers.sort_values(by="Multiplier").round(2)

,Team,Multiplier
25,Detroit Red Wings,0.72
16,Washington Capitals,0.73
22,Vancouver Canucks,0.75
14,Boston Bruins,0.76
17,Anaheim Ducks,0.77
15,Toronto Maple Leafs,0.83
10,Montreal Canadiens,0.83
4,Tampa Bay Lightning,0.84
18,Nashville Predators,0.84
6,Columbus Blue Jackets,0.87


In [25]:
games_this_week

{'Minnesota Wild': 3,
 'Colorado Avalanche': 4,
 'Tampa Bay Lightning': 2,
 'Dallas Stars': 4,
 'St. Louis Blues': 2,
 'Ottawa Senators': 3,
 'Columbus Blue Jackets': 3,
 'Edmonton Oilers': 4,
 'New Jersey Devils': 4,
 'Buffalo Sabres': 3,
 'Montreal Canadiens': 3,
 'Vegas Golden Knights': 3,
 'Carolina Hurricanes': 2,
 'New York Islanders': 3,
 'Boston Bruins': 3,
 'Toronto Maple Leafs': 2,
 'Washington Capitals': 3,
 'Anaheim Ducks': 4,
 'Nashville Predators': 3,
 'Chicago Blackhawks': 3,
 'Utah Hockey Club': 3,
 'Vancouver Canucks': 2,
 'Calgary Flames': 3,
 'Seattle Kraken': 4,
 'Pittsburgh Penguins': 4,
 'Detroit Red Wings': 3,
 'Los Angeles Kings': 4,
 'Florida Panthers': 4,
 'San Jose Sharks': 4,
 'Philadelphia Flyers': 3,
 'New York Rangers': 3,
 'Winnipeg Jets': 2}

In [26]:
injuries_df

,Player,Position,Updated,Injury,Injury Status,Team
0,Isac Lundestrom,C,"Thu, Oct 10",Lower Body,Day-to-Day,Anaheim
1,John Gibson,G,"Mon, Oct 7",Abdomen,IR. Expected to be out until at least Nov 8,Anaheim
2,Alec Regula,D,"Sun, Oct 6",Knee,IR. Expected to be out until at least Oct 16,Boston
3,John-Jason Peterka,RW,"Sat, Oct 12",Concussion,Expected to be out until at least Oct 16,Buffalo
4,Nicolas Aube-Kubel,RW,"Tue, Oct 8",Lower Body,IR. Expected to be out until at least Nov 1,Buffalo
...,...,...,...,...,...,...
79,T.J. Oshie,RW,"Tue, Sep 24",Back,IR. Expected to be out until at least Sep 15,Washington
80,Nicklas Backstrom,C,"Tue, Apr 30",Hip,IR. Expected to be out until at least Jul 1,Washington
81,David Gustafsson,C,"Sat, Oct 12",Lower Body,Day-to-Day,Winnipeg
82,Ville Heinola,D,"Mon, Oct 7",Ankle,IR. Expected to be out until at least Oct 24,Winnipeg


In [33]:
team_df = projections_df[projections_df["Team"] == 'Utah Hockey Club']
team_df

,Team,Injured,Player,Position,games_this_week,proj_fantasy_pts
